***
$\mathbf{\text{Assignment}}$<br>
***

This assignment is based on the concept of Type Stable Functions in Julia. Your task is to create two different implementations of the function $\mathbf{\text{Solve(V1,V2)}}$ which takes as input two Vectors of the same size and computes the following sum $\sum \limits _{i=1} ^{N} f(V1[i], V2[i])$.<br> $f(x,y)$ is defined as $4x^4 + 9x^2 + 10x + 1$ if x>y and $y^4 + 3y^2 + 9y + 2$ otherwise. <br>
For the first implementation create a function $Solve1(V1,V2)$ and do all the computations in that function itself. For the second implementation create a function $Solve2(V1,V2)$ which uses $\mathbf{\text{type stable functions}}$ inside it to do the computations. Check whether $Solve1$ and $Solve2$ are type-stable and compare the performance of the two implementations.

In [1]:
#### Helper Code #####
using BenchmarkTools

function genRandInt()
	return rand(1:100)
end

function genRandFloat()
	num = rand(Float64)*100.0
	while num < 1
		num = rand(Float64)*100.0
	end
	return num
end

function fillValues(V::Vector{Any}, n::Int)
	for i in 1:n
		choice = rand(Bool)
		if choice
			V[i] = genRandInt()
		else
			V[i] = genRandFloat()
		end
	end
end

fillValues (generic function with 1 method)

In [2]:
V1 = Vector{Any}(undef,10000)
V2 = Vector{Any}(undef,10000)
# Populate these 2 vectors with random float and integer values (in the range 1-100) in a ratio close to 1:1. Try to 
# make the Vectors as randomised as possible.
vsize = 10000
fillValues(V1,vsize)
fillValues(V2,vsize)

In [3]:
function Solve1(V1,V2)
	Sum = 0.0
	for i in 1:size(V1)[1]
		@inbounds x = V1[i]
		@inbounds y = V2[i]
		if x > y
			Sum += 4x^4+9x^2+10x+1
		else
			Sum += y^4+3y^2+9y+2
		end
	end
	return Sum
end

Solve1 (generic function with 1 method)

In [4]:
function f(x,y)  #### Type-Stable function for computation
	tx,ty= promote(x,y)
	if tx > ty
		return 4tx^4+9tx^2+10tx+1
	else
		return ty^4+3ty^2+9ty+2
	end
end

function Solve2(V1,V2)
	Sum = 0.0
	for i in 1:size(V1)[1]
		@inbounds x = V1[i]
		@inbounds y = V2[i]
		Sum += f(x,y)
	end
	return Sum
end

Solve2 (generic function with 1 method)

In [5]:
println("Runtime for Solve1: ")
println(@btime Solve1(V1,V2))
println("")
println("Runtime for Solve2: ")
println(@btime Solve2(V1,V2))

Runtime for Solve1: 


  5.033 ms (83197

 allocations: 1.27 MiB)


8.440991239530161e11

Runtime for Solve2: 


  1.110 ms (19996 allocations: 312.44 KiB)
8.440991239530161e11


In [6]:
####(1.) If f(x,y) was made to always return a Float64 Type, Good improvement.
# function f(x,y)
# 	if x > y
# 		return Float64(4x^4+9x^2+10x+1)
# 	else
# 		return Float64(y^4+3y^2+9y+2)
# 	end
# end

####(2.) If Solve2 was to be made type-stable using a helper function, Good improvement
# function f(x,y)
# 	tx,ty= promote(x,y)
# 	if tx > ty
# 		return 4tx^4+9tx^2+10tx+1
# 	else
# 		return ty^4+3ty^2+9ty+2
# 	end
# end

# function convert(V)
#     for i::Int64 in 1:size(V)[1]
#             if typeof(V[i]) != Float64
#                 V[i] = convert(Float64,V[i])
#             end
#         end
#     return V
# end

####(3.) Since the values to be evaluated are single-variable functions, another way would be to define two new functions(one for x,one for y)
# for each condition. Both of can be type-annotated for Int64, Float64 giving 4 functions in total.

# function fX(x::Int64)
#     return 4x^4+9x^2+10x+1
# end
# function fX(x::Float64)
#     return 4x^4+9x^2+10x+1
# end
# function fY(y::Int64)
#     return y^4+3y^2+9y+2
# end
# function fY(y::Float64)
#     return y^4+3y^2+9y+2
# end

#Best Improvement

